<a href="https://colab.research.google.com/github/IamToef/NCKH/blob/main/FaceRegconizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets==2.18.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch .nn as nn

from PIL import Image
from datasets import load_dataset
from torch.utils.data import Dataset , DataLoader
from torchvision.models import resnet18
from torchvision import transforms

In [ ]:
DATASET_NAME = 'cats_vs_dogs'
datasets = load_dataset(DATASET_NAME)
datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for cats_vs_dogs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cats_vs_dogs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(

Generating train split:   0%|          | 0/23410 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 23410
    })
})

In [ ]:
TEST_SIZE = 0.2
datasets = datasets['train'].train_test_split (test_size = TEST_SIZE )

**(a) Xây dựng hàm tiền xử lý dữ liệu hình ảnh**

In [ ]:
IMG_SIZE = 64
img_transforms = transforms.Compose ([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

**(b) Xây dựng class CatDogDataset**

In [ ]:
class CatDogDataset(Dataset):
  def __init__(self, data, transform=None):
    self.data = data
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    images = self.data[idx]['image']
    labels = self.data[idx]['labels']

    if self.transform:
      images = self.transform(images)

    labels = torch.tensor(labels, dtype=torch.long)

    return images, labels

**(c) Khai báo dataloader**

In [ ]:
TRAIN_BATCH_SIZE = 512
VAL_BATCH_SIZE = 256

train_dataset = CatDogDataset ( datasets ['train'], transform = img_transforms )
test_dataset = CatDogDataset ( datasets ['test'], transform = img_transforms )
train_loader = DataLoader ( train_dataset , batch_size = TRAIN_BATCH_SIZE , shuffle = True )
test_loader = DataLoader ( test_dataset , batch_size = VAL_BATCH_SIZE , shuffle = False )

**5. Xây dựng mô hình**

In [ ]:
class CatDogModel(nn.Module):
  def __init__ (self, n_classes):
    super(CatDogModel, self).__init__()

    resnet_model = resnet18(weights='IMAGENET1K_V1')
    self.backbone = nn.Sequential(*list(resnet_model.children())[:-1])
    for param in self.backbone.parameters():
      param.requires_grad = False

    in_features = resnet_model.fc.in_features
    self.fc = nn.Linear(in_features, n_classes)

  def forward (self, x):
    x = self.backbone(x)
    x = torch.flatten(x, 1)
    x = self.fc(x)

    return x

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
N_CLASSES = 2
model = CatDogModel(N_CLASSES).to(device)
test_input = torch.rand(1, 3, 224, 224).to(device)
with torch.no_grad():
  output = model(test_input)
  print(output.shape)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 43.4MB/s]


torch.Size([1, 2])


In [ ]:
EPOCH = 10
LR = 1e-3
WEIGHT_DECAY= 1e-5

optimizer = torch.optim.Adam(model.parameters(), lr = LR, weight_decay = WEIGHT_DECAY)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(EPOCH):
  train_losses = []
  model.train()
  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)

    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_losses.append(loss.item())

  train_loss = sum(train_losses)/len(train_losses)

  val_losses = []
  model.eval()
  with torch.no_grad():
    for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      loss = criterion(outputs, labels)

      val_losses.append(loss.item())

  val_loss = sum(val_losses)/len(val_losses)

  print(f'EPOCH {epoch + 1}:\tTrain loss: {train_loss: .3f}\tVal loss: {val_loss: .3f}')

EPOCH 1:	Train loss:  0.636	Val loss:  0.600
EPOCH 2:	Train loss:  0.545	Val loss:  0.534
EPOCH 3:	Train loss:  0.526	Val loss:  0.527
EPOCH 4:	Train loss:  0.517	Val loss:  0.523
EPOCH 5:	Train loss:  0.513	Val loss:  0.521
EPOCH 6:	Train loss:  0.509	Val loss:  0.519
EPOCH 7:	Train loss:  0.506	Val loss:  0.524
EPOCH 8:	Train loss:  0.505	Val loss:  0.523
EPOCH 9:	Train loss:  0.506	Val loss:  0.520
EPOCH 10:	Train loss:  0.505	Val loss:  0.524


In [ ]:
SAVE_PATH = 'cat_dog_weights.pt'
torch.save(model.state_dict(), SAVE_PATH)